**Most of a program's run time is spent inside loops**

Mostly function calls inside loops.

**The act of calling these functions itself better be fast!**

### But it doesn't look like it should be fast!

To execute `foo(x,y)`, Julia chooses the **method for foo that is _most_ specific to types of x, and y**.

**Looking up this method can be expensive!** There are hundreds of methods for `+(x,y)` picking the most specific one can involve a lot of head-scratching.

But Julia can do it when **compiling** the code, once and forever!

In [7]:
function sum1(xs)
    s = zero(xs[1])

    for i=1:length(xs)
        s = +(s, xs[i])
    end
    s
end

sum1 (generic function with 1 method)

In [8]:
@code_typed optimize=false sum1([1+2im, 7+9im])

CodeInfo(
│2 1 ─ %1  = (Base.getindex)(xs, 1)::Complex{Int64}
│  │         (s = (Main.zero)(%1))::Complex{Int64}
│4 │   %3  = (Main.length)(xs)::Int64
│  │   %4  = (1:%3)::UnitRange{Int64}
│  │         (#temp# = (Base.iterate)(%4))::Union{Nothing, Tuple{Int64,Int64}}
│  │   %6  = (#temp# === nothing)::Bool
│  │   %7  = (Base.not_int)(%6)::Bool
│  └──       goto #4 if not %7
│  2 ┄ %9  = #temp#::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│  │         (i = (Core.getfield)(%9, 1))::Int64
│  │   %11 = (Core.getfield)(%9, 2)::Int64
│5 │   %12 = s::Complex{Int64}
│  │   %13 = (Base.getindex)(xs, i)::Complex{Int64}
│  │         (s = %12 + %13)::Complex{Int64}
│  │         (#temp# = (Base.iterate)(%4, %11))::Union{Nothing, Tuple{Int64,Int64}}
│  │   %16 = (#temp# === nothing)::Bool
│  │   %17 = (Base.not_int)(%16)::Bool
│  └──       goto #4 if not %17
│  3 ─       goto #2
│7 4 ─       return s
) => Complex{Int64}

In [9]:
@code_typed sum1([1+2im, 7+9im]) # Also gets inlined -- no call to + on Complex anymore!

CodeInfo(
│╻     getindex2 1 ──       (Base.arrayref)(true, xs, 1)::Complex{Int64}
││╻╷╷╷  oftype  │    %2  = %new(Complex{Int64}, 0, 0)::Complex{Int64}
│╻     length4 │    %3  = (Base.arraylen)(xs)::Int64
││╻╷╷╷  Type  │    %4  = (Base.sle_int)(1, %3)::Bool
│││╻     unitrange_last  │          (Base.sub_int)(%3, 1)::Int64
││││    │    %6  = (Base.ifelse)(%4, %3, 0)::Int64
││╻╷╷   isempty  │    %7  = (Base.slt_int)(%6, 1)::Bool
││      └───       goto #3 if not %7
││      2 ──       goto #4
││      3 ──       goto #4
│       4 ┄─ %11 = φ (#2 => true, #3 => false)::Bool
│       │    %12 = φ (#3 => 1)::Int64
│       │    %13 = φ (#3 => 1)::Int64
│       │    %14 = (Base.not_int)(%11)::Bool
│       └───       goto #10 if not %14
│       5 ┄─ %16 = φ (#4 => 0, #9 => %22)::Int64
│       │    %17 = φ (#4 => 0, #9 => %24)::Int64
│       │    %18 = φ (#4 => %12, #9 => %31)::Int64
│       │    %19 = φ (#4 => %13, #9 => %32)::Int64
│╻     getindex5 │    %20 = (Base.arrayref)(true, xs, %18)::Compl

In [15]:
@noinline add(x,y) = x+y # forcefully avoid inlining

add (generic function with 1 method)

In [18]:
function sum2(xs)
    s = zero(xs[1])

    for i=1:length(xs)
        s = (s + xs[i])
    end
    s
end

sum2 (generic function with 1 method)

In [20]:
@code_native sum2([1+1im, 2+3im])

	.text
; Function sum2 {
; Location: In[18]:2
	pushq	%rbp
	movq	%rsp, %rbp
; Function getindex; {
; Location: array.jl:731
	movq	8(%rsi), %rax
	testq	%rax, %rax
	je	L107
;}
; Location: In[18]:4
	jle	L91
	movq	(%rsi), %rdx
	vpxor	%xmm0, %xmm0, %xmm0
	movl	$1, %ecx
	nopl	(%rax,%rax)
; Location: In[18]:5
; Function +; {
; Location: complex.jl:266
; Function +; {
; Location: int.jl:53
L32:
	vpaddq	(%rdx), %xmm0, %xmm0
;}}
; Function iterate; {
; Location: range.jl:575
; Function ==; {
; Location: promotion.jl:425
	cmpq	%rcx, %rax
;}}
	je	L95
; Function getindex; {
; Location: array.jl:731
	addq	$16, %rdx
	cmpq	%rax, %rcx
;}
; Function iterate; {
; Location: range.jl:576
; Function +; {
; Location: int.jl:53
	leaq	1(%rcx), %rcx
;}}
; Function getindex; {
; Location: array.jl:731
	jb	L32
	movq	%rsp, %rdx
	leaq	-16(%rdx), %rax
	movq	%rax, %rsp
	movq	%rcx, -16(%rdx)
	movabsq	$jl_bounds_error_ints, %rcx
	movl	$1, %edx
	movq	%rsi, %rdi
	movq	%rax, %rsi
	callq	*%rcx
;}}
; Function sum2 {
; Locati

Let's try to create a function that doesn't allow you to infer its result type

In [22]:
vec1 = [1,2]
@show typeof(vec1)

vec2 = Union{Int, Float64}[1,2.3]
@show typeof(vec2)

vec3 = [1,"hi"]
@show typeof(vec3)
vec3[1] # what is the output type of vec[i] for all i?


function bad_add(x,y)
    d=[0, "yes"] # intentionally create a Vector{Any}
    x+y+d[1]
end

typeof(vec1) = Array{Int64,1}
typeof(vec2) = Array{Union{Float64, Int64},1}
typeof(vec3) = Array{Any,1}


bad_add (generic function with 1 method)

In [23]:
@code_typed bad_add(1,2)

CodeInfo(
│  13 1 ─ %1 = invoke Base.vect(0::Int64, "yes"::Vararg{Any,N} where N)::Array{Any,1}
│╻  getindex14 │   %2 = (Base.arrayref)(true, %1, 1)::Any
││╻  +   │   %3 = (Base.add_int)(x, y)::Int64
││    │   %4 = (%3 + %2)::Any
│     └──      return %4
) => Any

In [24]:


function sum3(xs)
    s = zero(xs[1])

    for i=1:length(xs)
        s = bad_add(s, xs[i])
    end
    s
end

sum3 (generic function with 1 method)

In [25]:
@code_typed bad_add(1,2)

CodeInfo(
│  13 1 ─ %1 = invoke Base.vect(0::Int64, "yes"::Vararg{Any,N} where N)::Array{Any,1}
│╻  getindex14 │   %2 = (Base.arrayref)(true, %1, 1)::Any
││╻  +   │   %3 = (Base.add_int)(x, y)::Int64
││    │   %4 = (%3 + %2)::Any
│     └──      return %4
) => Any

In [26]:
@code_warntype sum3([1,2])

Body::Any
│╻     getindex4 1 ──       (Base.arrayref)(true, xs, 1)
│╻     length6 │    %2  = (Base.arraylen)(xs)::Int64
││╻╷╷╷  Type  │    %3  = (Base.sle_int)(1, %2)::Bool
│││╻     unitrange_last  │          (Base.sub_int)(%2, 1)
││││    │    %5  = (Base.ifelse)(%3, %2, 0)::Int64
││╻╷╷   isempty  │    %6  = (Base.slt_int)(%5, 1)::Bool
││      └───       goto #3 if not %6
││      2 ──       goto #4
││      3 ──       goto #4
│       4 ┄─ %10 = φ (#2 => true, #3 => false)::Bool
│       │    %11 = φ (#3 => 1)::Int64
│       │    %12 = φ (#3 => 1)::Int64
│       │    %13 = (Base.not_int)(%10)::Bool
│       └───       goto #10 if not %13
│       5 ┄─ %15 = φ (#4 => 0, #9 => %19)::Any
│       │    %16 = φ (#4 => %11, #9 => %25)::Int64
│       │    %17 = φ (#4 => %12, #9 => %26)::Int64
│╻     getindex7 │    %18 = (Base.arrayref)(true, xs, %16)::Int64
│       │    %19 = (Main.bad_add)(%15, %18)::Any
││╻     ==  │    %20 = (%17 === %5)::Bool
││      └───       goto #7 if not %20
││      6 ──  

### Common pitfall Number 1. use of global variables in loops

In [35]:
xs = rand(10^6)
function sum1_global()
    s = zero(xs[1])
    
    for i=1:length(xs)
        s = +(s, xs[i])
    end
    s
end


sum1_global (generic function with 1 method)

In [36]:
@code_warntype sum1_global()

Body::Any
│    3 1 ─ %1  = (Base.getindex)(Main.xs, 1)::Any
│      │   %2  = (Main.zero)(%1)::Any
│    5 │   %3  = (Main.length)(Main.xs)::Any
│      │   %4  = (isa)(%3, Int64)::Bool
│      └──       goto #3 if not %4
│      2 ─ %6  = π (%3, Int64)
│╻╷╷╷ Colon  │   %7  = (Base.sle_int)(1, %6)::Bool
││╻    Type  │         (Base.sub_int)(%6, 1)
│││┃│   unitrange_last  │   %9  = (Base.sub_int)(1, 1)::Int64
││││   │   %10 = (Base.ifelse)(%7, %6, %9)::Int64
│││    │   %11 = %new(UnitRange{Int64}, 1, %10)::UnitRange{Int64}
│      └──       goto #4
│      3 ─ %13 = (1:%3)::Any
│      └──       goto #4
│      4 ┄ %15 = φ (#2 => %11, #3 => %13)::Any
│      │   %16 = (Base.iterate)(%15)::Any
│      │   %17 = (%16 === nothing)::Bool
│      │   %18 = (Base.not_int)(%17)::Bool
│      └──       goto #7 if not %18
│      5 ┄ %20 = φ (#4 => %2, #6 => %25)::Any
│      │   %21 = φ (#4 => %16, #6 => %26)::Any
│      │   %22 = (Core.getfield)(%21, 1)::Any
│      │   %23 = (Core.getfield)(%21, 2)::Any
│   

### Fixes
- Make the global an argument!
- Make the global a `const`

In [37]:
const arr = rand(10^6)
function sum2_global()
    s = zero(arr[1])
    
    for i=1:length(arr)
        s = s + arr[i]
    end
    s
end

sum2_global (generic function with 1 method)

In [39]:
@time sum1_global()
@time sum2_global()

  0.128758 seconds (5.00 M allocations: 91.537 MiB, 10.37% gc time)
  0.001536 seconds (5 allocations: 176 bytes)


499821.06931097474

**Let's practice propagating type information in our head**

Run through the program and find out the types assigned to each variable over time.

In [41]:
function logint2(n)
    r = 0
    while n > 1
        n = n/2
        r += 1
    end
    r
end

logint2 (generic function with 1 method)

In [32]:
function logint2(n) # assume n::Float64
    r = 0 # Int
    while n > 1 # n :: Float64
        n = n/2 # n :: Float64
        r += 1 # Int
    end
    r # Int
end

logint2 (generic function with 1 method)

In [42]:
@code_typed logint2(10.0)

CodeInfo(
│     1 ─       nothing::Nothing
│   3 2 ┄ %2  = φ (#1 => 0, #3 => %12)::Int64
│     │   %3  = φ (#1 => _2, #3 => %11)::Float64
│╻   >  │   %4  = π (1.0, Float64)
││┃│  <  │   %5  = (Base.lt_float)(%4, %3)::Bool
│││   │   %6  = π (1.0, Float64)
│││╻   ==  │   %7  = (Base.eq_float)(%6, %3)::Bool
│││╻   &  │   %8  = (Base.and_int)(%7, false)::Bool
│││╻   |  │   %9  = (Base.or_int)(%5, %8)::Bool
│     └──       goto #4 if not %9
│╻╷  /4 3 ─ %11 = (Base.div_float)(%3, 2.0)::Float64
│╻   +5 │   %12 = (Base.add_int)(%2, 1)::Int64
│     └──       goto #2
│   7 4 ─       return %2
) => Int64

In [51]:
div(900.0,7)

128.0

In [57]:
function logint2(n) # assume n::Int
    r = 0 # Int
    n = Float64(n)
    while n > 1 # n :: Union{Int, Float64} (propagated from below)
        n = n/2 # n :: Union{Int, Float64}
        r += 1 # Int
    end
    r # Int
end

logint2 (generic function with 1 method)

In [58]:
@code_typed logint2(10)

CodeInfo(
│   3 1 ─ %1  = π (n, Int64)
│╻   Type  └── %2  = (Base.sitofp)(Float64, %1)::Float64
│   4 2 ┄ %3  = φ (#1 => 0, #3 => %15)::Int64
│     │   %4  = φ (#1 => %2, #3 => %14)::Float64
│     │   %5  = π (%4, Float64)
│╻   >  │   %6  = π (1.0, Float64)
││┃│  <  │   %7  = (Base.lt_float)(%6, %5)::Bool
│││   │   %8  = π (1.0, Float64)
│││╻   ==  │   %9  = (Base.eq_float)(%8, %5)::Bool
│││╻   &  │   %10 = (Base.and_int)(%9, false)::Bool
│││╻   |  │   %11 = (Base.or_int)(%7, %10)::Bool
│     └──       goto #4 if not %11
│   5 3 ─ %13 = π (%4, Float64)
││╻   /  │   %14 = (Base.div_float)(%13, 2.0)::Float64
│╻   +6 │   %15 = (Base.add_int)(%3, 1)::Int64
│     └──       goto #2
│   8 4 ─       return %3
) => Int64

In [59]:
function sumlog(xs)
    s = zero(xs[1])
    
    for i=1:length(xs)
        s = s + logint2(xs[i]) # If type of xs[i] is known, result of logint2 will be known
    end
    s
end

sumlog (generic function with 1 method)

In [60]:
@code_typed optimize=false sumlog(xs)

CodeInfo(
│2 1 ─ %1  = (Base.getindex)(xs, 1)::Float64
│  │         (s = (Main.zero)(%1))::Const(0.0, false)
│4 │   %3  = (Main.length)(xs)::Int64
│  │   %4  = (1:%3)::UnitRange{Int64}
│  │         (#temp# = (Base.iterate)(%4))::Union{Nothing, Tuple{Int64,Int64}}
│  │   %6  = (#temp# === nothing)::Bool
│  │   %7  = (Base.not_int)(%6)::Bool
│  └──       goto #4 if not %7
│  2 ┄ %9  = #temp#::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│  │         (i = (Core.getfield)(%9, 1))::Int64
│  │   %11 = (Core.getfield)(%9, 2)::Int64
│5 │   %12 = s::Float64
│  │   %13 = (Base.getindex)(xs, i)::Float64
│  │   %14 = (Main.logint2)(%13)::Int64
│  │         (s = %12 + %14)::Float64
│  │         (#temp# = (Base.iterate)(%4, %11))::Union{Nothing, Tuple{Int64,Int64}}
│  │   %17 = (#temp# === nothing)::Bool
│  │   %18 = (Base.not_int)(%17)::Bool
│  └──       goto #4 if not %18
│  3 ─       goto #2
│7 4 ─       return s
) => Float64

In [61]:
@time logint2.([i for i in 1:2^20]);

  0.112937 seconds (257.51 k allocations: 28.957 MiB, 3.63% gc time)


# Summary

**Uncertainty** is the root cause of slowness.

It can come in many forms!

1. Globals -- uncertainty about type after compilation
2. Type instability -- uncertainty about the input or output type
3. Not fixed length -- arrays or strings don't have a fixed size
3. Mutability -- uncertainty that someone else may mutate the object
4. Boxing -- uncertainty about type or, size of values

Use `@time` and pay attention to allocations -- uncertainty leads to boxing which leads to GC overhead, so you will see it!

## References

- Performance tips from the manual https://docs.julialang.org/en/v0.6.1/manual/performance-tips/
- Engineering Julia for Speed - Lionel Zoubritzky
 https://www.youtube.com/watch?v=XWIZ_dCO6X8
- Information overload: tools for making program analysis and debugging manageable - Jameson Nash JuliaCon 2018